#### Declare necessary modules/links/directories

In [38]:
from bs4 import BeautifulSoup
import requests
import re
import json
import scrapping_functions as sf
reload(sf)
from selenium import webdriver
#Start = signifies the listing to start it, increases in increments of 10 per page
#End = 990
target_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Washington+DC&start='
base = 'https://www.yelp.com/'

In [45]:
#Pull in a list of links from the target url
link_dict = {}
for x in range(10, 990,10):
    target = target_url + str(x)
    raw_html = requests.get(target)
    soup = BeautifulSoup(raw_html.text, 'html.parser')
    link_dict = sf.biz_links(soup, link_dict)


'4.5 '

In [3]:
###Don't need to do this anymore! We're only keeping links with valid reviews
#Remove extraneous links, keep only business links
for link in link_list:
    if re.search("adredir", link):
        link_list.pop(link_list.index(link))

In [48]:
#Write all links to a text file
biz_links = open('cleanbiz_links.txt', 'w')
for item in link_dict.keys():
    biz_links.write("%s\n" % item)
biz_links.close()

We would like to construct the following dictionary:

biz_dict = {biz_name: {"city": "Washington", "state": "DC", "category_aliases": "newamerican,breakfast_brunch", "biz_id": "wO-7cBBOYUdiLflpuRsu9A", "latitude": 38.90842, "biz_name": "The Bird", "city_state": "Washington, DC", "longitude": -77.026685, "geoquad": 12845454}}

unique_id = 5D32F13B349CE2AD

#### Algorithm design:

For each link in biz_links:
    > set biz_name = replace("_", '-' in link)
    > find the unique_id
    > assign biz_dict[biz_name] = dictionary(biz_name)

In [56]:
link_file = open("cleanbiz_links.txt", "r")
link_list = link_file.read().split('\n')
link_list = list(set(link_list))
for link in link_list:
    if link == '':
        link_list.pop(link_list.index(link))

In [84]:
biz_dict = {}

for biz_name in link_list:
    #biz_name = re.sub("-washington*", '', biz_name)
    biz_dict[biz_name] = {}
    
    raw_html = requests.get(base + biz_name)
    soup = BeautifulSoup(raw_html.text, 'html.parser')
    parse_string = str(soup.find_all("script"))
    parse_list = parse_string.split('<script>')
    
    try:
        match = re.sub(".*,null,",'', [s for s in parse_list if re.search(".*latitude.*", s)][0])
    except IndexError:
        "The regexp is matching too many JSON entries, please take a look at " + str(link)
    
    biz_dict[biz_name] = json.loads(re.sub(", \"geoquad\".*",'', match) + '}')
    biz_dict[biz_name]['rating'] = link_dict[biz_name].replace(" ", "")

In [88]:
print biz_dict[biz_dict.keys()[0]]
print biz_dict.keys()[0]

{u'city': u'Washington', 'rating': '4.5', u'category_aliases': u'foodtrucks,korean,tradamerican', u'biz_name': u'Rolling Cow', u'city_state': u'Washington, DC', u'longitude': -77.0413344, u'state': u'DC', u'biz_id': u'ynuv5y74SB2gEXFPjMYb_Q', u'latitude': 38.897244}
/biz/rolling-cow-washington


In [92]:
#Output JSON file of all the business details
with open('biz_details.json', 'w') as outfile:
    json.dump(biz_dict, outfile)

In [91]:
biz_json = json.load(open("/Users/robertsonwang/Desktop/Python/Yelp/Yelp_scrapper/biz_details.json"))
agg_dict = dict(biz_json, **biz_dict)
with open('biz_details_agg.json', 'w') as outfile:
    json.dump(agg_dict, outfile)

### Testing is going on beyond this point

In [36]:
# test_link = '10'
# target_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Washington+DC&start='

# target = target_url + test_link
# raw_html = requests.get(target)
# soup = BeautifulSoup(raw_html.text, 'html.parser')

articles = soup.find_all("div", class_="biz-listing-large")
link_dict = {}
for article in articles:
    rating_match = re.search(r'(.*) star rating', str(article.find_all('div', class_ = "biz-rating biz-rating-large clearfix")))
    link_match = re.search(r'href=[\'"]?([^\'" >]+)', str(article.find_all(href = True)))
    if link_match and rating_match:
        link = link_match.group(0)
        link = link[6:]
        link = link.replace('?osq=Restaurants', '')
        rating = rating_match.group(0)[len(rating_match.group(0))-15:]
        rating = rating.replace("star rating", "")
        link_dict[link] = rating
    
link_dict

{'/biz/convivial-washington': '4.0 ',
 '/biz/fiola-mare-washington': '4.0 ',
 '/biz/honeysuckle-washington-3': '4.5 ',
 '/biz/lemon-cuisine-of-india-washington': '4.5 ',
 '/biz/marcels-by-robert-wiedmaier-washington': '4.5 ',
 '/biz/mezcalero-washington': '4.0 ',
 '/biz/old-ebbitt-grill-washington': '4.0 ',
 '/biz/the-dabney-washington': '4.5 ',
 '/biz/the-pig-washington-2': '4.0 ',
 '/biz/zaytinya-washington': '4.0 '}

In [39]:
link_dict = {}
link_dict = sf.biz_links(soup, link_dict)

In [40]:
link_dict

{'/biz/convivial-washington': '4.0 ',
 '/biz/fiola-mare-washington': '4.0 ',
 '/biz/honeysuckle-washington-3': '4.5 ',
 '/biz/lemon-cuisine-of-india-washington': '4.5 ',
 '/biz/marcels-by-robert-wiedmaier-washington': '4.5 ',
 '/biz/mezcalero-washington': '4.0 ',
 '/biz/old-ebbitt-grill-washington': '4.0 ',
 '/biz/the-dabney-washington': '4.5 ',
 '/biz/the-pig-washington-2': '4.0 ',
 '/biz/zaytinya-washington': '4.0 '}

WE CAN DEFINE DISTANCE BETWEEN (CITY, REVIEW) AS COSINE SIMILARITY!